In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt import EfficientFrontier, risk_models, expected_returns, objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import warnings
warnings.filterwarnings("ignore")


In [2]:
data=pd.read_excel("Stock List.xlsx")
data
remove_stock=['OSG','MSTR']
data=data[~data['Ticker'].isin(remove_stock)] 
# data=data[data['Country']=='USA']
# data=data.iloc[:1000]
data.keys()

Index(['Ticker', 'Name', 'Sector', 'Country', 'Mcap'], dtype='object')

In [3]:
All_Tickers=data.Ticker.to_list()
All_Tickers = [ticker for ticker in All_Tickers if isinstance(ticker, str)]
sectors = data['Sector'].to_list()


In [4]:
prices_data = yf.download(All_Tickers, start='2023-01-01')['Adj Close']


[*********************100%%**********************]  4230 of 4230 completed

16 Failed downloads:
['PWFL', 'WRK', 'CVLY', 'ASLN', 'TNP', 'JAN', 'EVBG', 'OLK', 'VERY', 'ASTR', 'LSDI', 'AKLI', 'WIRE', 'ETRN', 'CATC', 'NWLI']: YFChartError('%ticker%: No data found, symbol may be delisted')


In [5]:
prices_data.to_csv("Price_Data.csv")

In [6]:
import datetime
start_date = datetime.datetime.today() - pd.DateOffset(months=6)
prices_train = prices_data.loc[start_date:]
df = prices_train.copy()
df = df.loc[:, ~(df < 1).any()]
# df=df.ffill()
df = df.dropna(axis=1)
df

Ticker,A,AA,AADI,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,...,ZS,ZTO,ZTS,ZUMZ,ZUO,ZURA,ZVRA,ZWS,ZYME,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-25,145.559998,31.860001,2.17,14.92,7.31,14.50,85.510002,84.250000,170.850006,21.340000,...,194.070007,21.090000,167.809998,14.220000,8.85,2.40,5.64,32.320000,10.40,12.74
2024-03-26,144.410004,31.580000,2.20,14.92,7.04,14.44,86.570000,84.250000,169.710007,21.090000,...,195.630005,21.030001,165.669998,14.460000,8.71,2.26,5.72,32.310001,10.47,12.35
2024-03-27,147.369995,33.209999,2.37,15.30,7.41,13.96,86.669998,85.320000,173.309998,21.639999,...,193.479996,21.270000,168.509995,14.800000,8.85,2.32,5.94,32.759998,10.30,12.58
2024-03-28,145.509995,33.790001,2.34,15.35,7.50,13.86,88.099998,85.089996,171.479996,21.910000,...,192.630005,20.940001,169.210007,15.190000,9.12,2.57,5.80,33.470001,10.52,12.37
2024-04-01,145.559998,34.020000,2.31,15.41,7.55,13.92,87.980003,85.430000,170.029999,21.120001,...,192.130005,21.450001,167.020004,14.800000,8.87,2.85,5.16,33.119999,9.88,12.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-17,138.350006,33.970001,1.84,11.17,10.06,13.58,96.180000,41.330002,216.789993,26.990000,...,170.320007,22.340000,195.710007,23.219999,8.83,3.88,7.53,33.799999,12.30,8.19
2024-09-18,139.009995,34.340000,1.87,11.03,10.05,13.28,96.930000,42.070000,220.690002,27.110001,...,168.520004,22.100000,196.479996,23.330000,8.74,4.13,7.83,34.020000,12.52,8.18
2024-09-19,141.949997,34.919998,1.83,11.11,10.04,14.20,102.449997,42.430000,228.869995,27.370001,...,172.580002,22.700001,193.240005,23.559999,8.80,4.30,8.02,34.980000,13.05,8.33


In [7]:
# mu = expected_returns.mean_historical_return(df)
# S = risk_models.risk_matrix(df, method='ledoit_wolf')
# ef = EfficientFrontier(mu, S,weight_bounds=(0, 0.4))
# i = 0
# while True:
#     try:
#         raw_weights = ef.efficient_return(target_return=mu.max()-i)
#         print(f"Selected i is {i}")
#         break
#     except Exception as e:
#         # print(f"Error for i={i}: {e}")
#         i += 0.01

# print(f"First successful i: {i}")
# print(f"Raw weights: {raw_weights}")
# # raw_weights=ef.max_sharpe()
# cleaned_weights = ef.clean_weights()
# ef.portfolio_performance(verbose=True)
# filtered_data = [(name, value) for name, value in cleaned_weights.items() if value > 0.1]
# print(filtered_data)

In [8]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.risk_matrix(df, method='ledoit_wolf')

# Initialize Efficient Frontier with sector constraints
sector_info = data.set_index('Ticker')['Sector'].to_dict()
sector_max_allocation = {sector: 0.4 for sector in set(sector_info.values())}
sector_map = {sector: [] for sector in sector_max_allocation.keys()}
for ticker in df.columns:
    sector = sector_info.get(ticker, None)
    if sector:
        sector_map[sector].append(ticker)

In [9]:
sector_map = {sector: [] for sector in sector_max_allocation.keys()}
for ticker in df.columns:
    sector = sector_info.get(ticker, None)
    if sector:
        sector_map[sector].append(ticker)

# Efficient Frontier
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))

# Add sector constraints directly
for sector, tickers in sector_map.items():
    indices = [df.columns.get_loc(ticker) for ticker in tickers if ticker in df.columns]
    if indices:
        ef.add_constraint(lambda w, indices=indices: sum(w[i] for i in indices) <= sector_max_allocation[sector])

# Optimize portfolio for max Sharpe ratio
raw_weights = ef.max_sharpe()
i = 0
while True:
    try:
        raw_weights = ef.efficient_return(target_return=mu.max()-i)
        print(f"Selected i is {i}")
        break
    except Exception as e:
        # print(f"Error for i={i}: {e}")
        i += 0.01
# Clean weights and display performance
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

# Filter non-zero weights
filtered_data = [(name, value) for name, value in cleaned_weights.items() if value > 0.01]
print(filtered_data)

Selected i is 41.090000000000394
Expected annual return: 11101.6%
Annual volatility: 65.0%
Sharpe Ratio: 170.79
[('ASTS', 0.28725), ('CMAX', 0.01438), ('CXAI', 0.04367), ('INSG', 0.09814), ('LASE', 0.38108), ('NEON', 0.27029), ('NEOV', 0.02005), ('SMMT', 0.03078), ('TIL', 0.06264), ('VVPR', 0.03), ('WGS', 0.03304), ('YIBO', 0.0291)]


In [10]:
pd.DataFrame(filtered_data).sort_values(by=1,ascending=False)

,0,1
4,LASE,0.38108
0,ASTS,0.28725
5,NEON,0.27029
3,INSG,0.09814
8,TIL,0.06264
2,CXAI,0.04367
10,WGS,0.03304
7,SMMT,0.03078
9,VVPR,0.03000
11,YIBO,0.02910


In [14]:
import tqdm
for i in tqdm.tqdm(range(20)):
    print(i)

100%|██████████| 20/20 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
